In [17]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torchvision.transforms.transforms import ToTensor

from models.model import My_siamese
from dataset.reconstruct import prepare_data
from dataset.loader import train_validation_loader
from dataset.loader import testset_loader

In [4]:
net = My_siamese().cuda()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0005)

data_dir = os.path.join("data")
datasets.Omniglot(data_dir, background=True, download=True, transform=ToTensor())
datasets.Omniglot(data_dir, background=False, download=True, transform=ToTensor())

Files already downloaded and verified
Files already downloaded and verified


Dataset Omniglot
    Number of datapoints: 0
    Root location: data/omniglot-py
    StandardTransform
Transform: ToTensor()

In [5]:
train_dir, val_dir, test_dir = prepare_data(data_dir, 3)
train_loader, val_loader = train_validation_loader(train_dir, val_dir, 16, False, 5, True, 3, 8)
test_loader = testset_loader(test_dir, 5, 3, 8)
    

In [9]:
net.load_state_dict(torch.load("./tutorial/siamese_omniglot_cnn/omniglot_cnn_best.pt")["model"])

<All keys matched successfully>

In [26]:
net.eval()
for i, (img1, img2, label) in enumerate(train_loader):
    output = net(img1.cuda(), img2.cuda())
    output = torch.sigmoid(output)+0.5
    loss = loss_fn(output, label.cuda())

    output = output.int()
    print("output : ", output)
    print("label : ", label)
    result = torch.sum(output == label.cuda())
    print("result : ", result)
    print("Accuray : ", result/img1.shape[0])


    break

output :  tensor([[1],
        [1],
        [1],
        [0],
        [1],
        [1],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [0],
        [1],
        [1],
        [0]], device='cuda:0', dtype=torch.int32)
label :  tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.]])
result :  tensor(13, device='cuda:0')
Accuray :  tensor(0.8125, device='cuda:0')


In [ ]:
with torch.no_grad():         
    for i, (img1, img2, label) in enumerate(val_loader):
        output = net(img1.cuda(), img2.cuda())